<a href="https://colab.research.google.com/github/nikpournastaran/Minerva-RAG-Grammar/blob/main/Minerva_RAG_Grammar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PHASE 1: THE VOICE ASSISTANT APP

1. INSTALL DEPENDENCIES

In [ ]:
!pip install -q gradio openai-whisper transformers accelerate bitsandbytes gtts langchain langchain-community langchain-core huggingface_hub

import gradio as gr
import whisper
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from gtts import gTTS
import os
import tempfile

2. CONFIGURATION

In [ ]:
LANG = "it"
WHISPER_MODEL_SIZE = "small"
LLM_MODEL_ID = "sapienzanlp/Minerva-7B-instruct-v1.0"
current_knowledge_base = "" # حافظه فایل‌های آپلود شده

DEFAULT_SYSTEM_PROMPT = """
You are an advanced Italian Grammar Assistant.
Your task is to analyze the user's spoken sentence based on the provided context (examples).
- If the input is a question with options (A/B), choose the correct one.
- If the input requires grammatical analysis (subject, verb, etc.), perform the analysis.
Keep the answer concise and strictly follow the format of the provided examples.
"""

3. LOAD MODELS

In [ ]:
print("⏳ Loading Whisper...")
whisper_model = whisper.load_model(WHISPER_MODEL_SIZE)

print("⏳ Loading Minerva LLM...")
quant_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=6.0)

try:
    tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_ID)
    model = AutoModelForCausalLM.from_pretrained(LLM_MODEL_ID, quantization_config=quant_config, device_map="auto")

    pipe = pipeline(
        "text-generation", model=model, tokenizer=tokenizer,
        max_new_tokens=256, temperature=0.3, top_p=0.9, do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    llm = HuggingFacePipeline(pipeline=pipe)
except Exception as e:
    print(f"Error: {e}")

4. HELPER FUNCTIONS

In [ ]:
def update_context(files):
    global current_knowledge_base
    context_text = ""
    if not files: return "No files."
    for file in files:
        with open(file.name, "r", encoding="utf-8") as f:
            context_text += f"\n--- {os.path.basename(file.name)} ---\n{f.read()}\n"
    current_knowledge_base = context_text
    return "✅ Knowledge Base Updated!"

def text_to_speech(text):
    if not text: return None
    tts = gTTS(text=text, lang="it")
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
        tts.save(fp.name)
        return fp.name

5. MAIN PIPELINE (RAG)

In [ ]:
def process_pipeline(audio_path):
    if audio_path is None: return "No audio.", None

    # A. Transcribe
    user_text = whisper_model.transcribe(audio_path, language="it")["text"]

    # B. Prepare Context (RAG)
    final_context = DEFAULT_SYSTEM_PROMPT
    if current_knowledge_base:
        final_context += f"\n\n### CONTEXT:\n{current_knowledge_base}"

    # C. Generate Answer
    prompt = f"Instructions:\n{final_context}\n\nUser:\n{user_text}\n\nAssistant (Italian):"
    full_response = llm.invoke(prompt)
    clean_response = full_response.replace(prompt, "").split("Assistant (Italian):")[-1].strip()

    # D. Speak
    audio_out = text_to_speech(clean_response)
    return f"🗣️ User: {user_text}\n🤖 AI: {clean_response}", audio_out

6. UI (GRADIO)

In [ ]:
with gr.Blocks(theme=gr.themes.Soft(), title="Italian AI Tutor") as demo:
    gr.Markdown("# 🇮🇹 AI Voice Assistant (Demo)")
    with gr.Row():
        with gr.Column():
            files = gr.File(label="Upload Grammar Rules (.txt)", file_count="multiple")
            btn_upd = gr.Button("Load Rules")
            status = gr.Textbox(label="Status")
            btn_upd.click(update_context, files, status)
        with gr.Column():
            mic = gr.Audio(sources=["microphone"], type="filepath")
            btn_run = gr.Button("Analyze")
            out_txt = gr.Markdown()
            out_aud = gr.Audio(autoplay=True)
            btn_run.click(process_pipeline, mic, [out_txt, out_aud])

demo.launch(share=True)

# PHASE 2: SCIENTIFIC EVALUATION (CSV Generation)

⚠️ IMPORTANT: Restart Runtime before running this!

In [ ]:
import json
import re
import csv
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

  DATA SETUP

In [ ]:
!pip install -q openai-whisper transformers accelerate bitsandbytes gtts langchain langchain-community langchain-core huggingface_hub pandas
print("✅ Installation complete.")

1. RAW DATA INPUT (Based on your files)

In [ ]:
RAW_TEXT_1 = """
---------------------------------------------------------Domanda 1----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Il papà è entusiato.
b. Il papà è entusiasta.
"b. Il papà è entusiasta."
---------------------------------------------------------Domanda 2----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Marco e Lucia sono molto belle.
b. Marco e Lucia sono molto belli.
"b. Marco e Lucia sono molto belli."
---------------------------------------------------------Domanda 3----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Il cane e il bambino correvano insieme.
b. Il cane e il bambino correva insieme.
"a. Il cane e il bambino correvano insieme."
---------------------------------------------------------Domanda 4----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Non so se al suo posto avrei fatto lo stesso.
b. Non so se al suo posto avessi fatto lo stesso.
"b. Non so se al suo posto avrei fatto lo stesso."
---------------------------------------------------------Domanda 5----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Quelli sono i libri della quale ti ho parlato.
b. Quelli sono i libri di cui ti ho parlato.
"b. Quelli sono i libri di cui ti ho parlato."
---------------------------------------------------------Domanda 6----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Ho incontrato Anna e gli ho dato un passaggio a casa.
b. Ho incontrato Anna e le ho dato un passaggio a casa.
"b. Ho incontrato Anna e le ho dato un passaggio a casa."
---------------------------------------------------------Domanda 7----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Ieri sono andato al cinema con mia madre.
b. Ieri ho andato al cinema con mia madre.
"La frase corretta è ""Ieri sono andato al cinema con mia madre."
---------------------------------------------------------Domanda 8----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a.  Il film che ho visto ieri mi è piaciuto molto.
b.  Il film il quale ho visto ieri mi è piaciuto molto.
"La frase corretta è: a. Il film che ho visto ieri mi è piaciuto molto.""
---------------------------------------------------------Domanda 9----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Quando Lucia e Marco arrivano, inizieremo a mangiare.
b. Quando Lucia e Marco arriva, inizieremo a mangiare."
"b. Quando Lucia e Marco arriva, inizieremo a mangiare."
---------------------------------------------------------Domanda 10----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Ieri sono andata al ristorante di cui mi hai parlato.
b. Ieri sono andata al ristorante che mi hai parlato.
"La frase corretta è la prima, ""Ieri sono andata al ristorante di cui mi hai parlato"
---------------------------------------------------------Domanda 11----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Sabato scorso di siamo divertito molto alla festa di Luca.
b. Sabato scorso di siamo divertiti molto alla festa di Luca.
"b. Sabato scorso ci siamo divertiti molto alla festa di Luca."
---------------------------------------------------------Domanda 12----------------------------------------------------------------------------------------------
1"Quale di queste due frasi è grammaticalmente corretta?
a. Spento la sveglia, mi sono alzata dal letto.
b. Spenta la sveglia, mi sono alzata dal letto."
"b. Spenta la sveglia, mi sono alzata dal letto."
---------------------------------------------------------Domanda 13----------------------------------------------------------------------------------------------
13,Quale di queste due frasi è grammaticalmente corretta?
a. Il sole splendeva gialla nel cielo.
b. Il sole splendeva giallo nel cielo.
"La risposta corretta è b. Il sole splendeva giallo nel cielo."
---------------------------------------------------------Domanda 14----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Ad Antonella le piacciono i film d'azione.
b. Ad Antonella piacciono i film d'azione.
"b. Ad Antonella piacciono i film d'azione."
---------------------------------------------------------Domanda 15----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Filippo si ha fatto male giocando a calcetto.
b. Filippo si è fatto male giocando a calcetto.
"b. Filippo si è fatto male giocando a calcetto."
---------------------------------------------------------Domanda 16----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Se volevo andare via l'avrei già fatto.
b. Se avessi voluto andare via l'avrei già fatto.
"b. Se avessi voluto andare via l'avrei già fatto."
---------------------------------------------------------Domanda 17----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Matteo, dopo aver salutato Luca, sono andati via.
b. Matteo, dopo aver salutato Luca, è andato via."
"b. Matteo, dopo aver salutato Luca, è andato via."
---------------------------------------------------------Domanda 18----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Ho letto il libro che mi hai consigliato.
b. Ho letto il libro di cui mi hai consigliato.
La frase corretta è: ""Ho letto il libro che mi hai consigliato."
---------------------------------------------------------Domanda 19----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Nella valle si sentiva un eco fortissimo.
b. Nella valle si sentiva un'eco fortissima.
"La frase corretta è la prima, "Nella valle si sentiva un eco fortissimo"
---------------------------------------------------------Domanda 20----------------------------------------------------------------------------------------------
Quale di queste due frasi è grammaticalmente corretta?
a. Il bambino ha rotto i giocattoli.
b. Il bambino ha rotti i giocattoli.
"b. Il bambino ha rotti i giocattoli."
"""

RAW_TEXT_2 = """
---------------------------------------------------------Domanda 1----------------------------------------------------------------------------------------------
Individua il soggetto nella frase seguente: Mi piace il gelato al cioccolato.
"Il soggetto è ""Mi piace"".
---------------------------------------------------------Domanda 2----------------------------------------------------------------------------------------------
Individua il soggetto nella frase seguente: Il gatto ha mangiato il topo.
"Il gatto è il soggetto della frase."
---------------------------------------------------------Domanda 3----------------------------------------------------------------------------------------------
Individua il soggetto nella frase seguente: Ieri ha piovuto tutta la giornata.
"Il soggetto è ""Ieri"".
---------------------------------------------------------Domanda 4----------------------------------------------------------------------------------------------
Individua il soggetto nella frase seguente: Anna si è fatta male cadendo dall'altalena.
"Il soggetto è ""Anna"".
---------------------------------------------------------Domanda 5----------------------------------------------------------------------------------------------
Individua il soggetto nella frase seguente: Chi ti ha detto di aprire la porta?
"Il soggetto è ""tu"" perché si riferisce alla persona che viene interrogata sulla questione.""
---------------------------------------------------------Domanda 6----------------------------------------------------------------------------------------------
Individua il soggetto nella frase seguente: A dirmelo è stato Luca.
"Luca."
---------------------------------------------------------Domanda 7----------------------------------------------------------------------------------------------
Individua il soggetto nella frase seguente: Oggi si mangia a casa
"Il soggetto è ""Oggi"" perché è la persona o l'oggetto che compie l'azione della frase.""
---------------------------------------------------------Domanda 8----------------------------------------------------------------------------------------------
Individua il soggetto della frase Solitamente mi lavo i capelli tre volte a settimana
"Il soggetto è ""mi"" (io).
---------------------------------------------------------Domanda 9----------------------------------------------------------------------------------------------
Individua il soggetto della frase: Chissà cosa è successo dopo
Il soggetto è ""cosa"".
---------------------------------------------------------Domanda 10----------------------------------------------------------------------------------------------
Individua il soggetto nella frase seguente: Ieri mi è capitata una cosa strana
"Il soggetto della frase è ""io"" perché la frase indica che l'azione viene compiuta da un individuo specifico, ovvero colui che parla o scrive.""
---------------------------------------------------------Domanda 11----------------------------------------------------------------------------------------------
"Indica quale funziona ha il ""che"" nella frase seguente: Che mi racconti di nuovo?"
"Il che è una congiunzione subordinante, quindi la risposta corretta è che funziona come subordinata alla frase principale ""mi racconti""."
---------------------------------------------------------Domanda 12----------------------------------------------------------------------------------------------
"Indica quale funziona ha il ""che"" nella frase seguente: Ieri ho incontrato Luca che mi ha raccontato del suo viaggio a Parigi."
"Il ""che"" nella frase è un pronome relativo che introduce una proposizione subordinata relativa al soggetto ""Luca"".
---------------------------------------------------------Domanda 13----------------------------------------------------------------------------------------------
"Indica quale funziona ha il ""che"" nella frase seguente: Stasera non posso fare troppo tardi che domani mi devo svegliare presto."
"Il ""che"" introduce una proposizione relativa (che specifica un termine della frase precedente) e quindi ha funzione di congiunzione subordinante.
---------------------------------------------------------Domanda 14----------------------------------------------------------------------------------------------
"Indica quale funziona ha il ""che"" nella frase seguente: Se non fossi riuscita a prendere l'ultimo treno non avrei saputo che fare."
"Il ""che"" introduce una proposizione relativa, quindi è la risposta corretta."
---------------------------------------------------------Domanda 15----------------------------------------------------------------------------------------------
"Indica quale funziona ha il ""che"" nella frase seguente: Che hai combinato!"
"Il che è una congiunzione subordinante che introduce una proposizione subordinata relativa, indicando un'azione o una situazione che si verifica dopo l'avverbio di tempo ""hai"".
---------------------------------------------------------Domanda 16----------------------------------------------------------------------------------------------
"Indica quale funziona ha il ""che"" nella frase seguente: Mi dispiace che te vada così presto."
"Il ""che"" in questa frase è una congiunzione subordinante che introduce la proposizione relativa ""Mi dispiace che"". Quindi, il ""che"" funziona come subordinata alla proposizione principale ""Mi dispiace che te vada così presto"".
---------------------------------------------------------Domanda 17----------------------------------------------------------------------------------------------
"Indica quale funziona ha il ""che"" nella frase seguente: Hai visto che bel giardino!"
"Il ""che"" è un pronome relativo"
---------------------------------------------------------Domanda 18----------------------------------------------------------------------------------------------
"Indica quale funziona ha il ""che"" nella frase seguente: Che hai fatto sabato sera?"
"Il ""che"" è una congiunzione subordinante che introduce la proposizione dipendente ""hai fatto sabato sera"". Quindi, la risposta è: ""Che hai fatto sabato sera?""
---------------------------------------------------------Domanda 19----------------------------------------------------------------------------------------------
"Indica quale funziona ha il ""che"" nella frase seguente: I film d'azione sono quelli che preferisco."
"Il ""che"" introduce una proposizione relativa, quindi la risposta è ""I film d'azione sono quelli che preferisco"".
---------------------------------------------------------Domanda 20----------------------------------------------------------------------------------------------
1"Indica quale funziona ha il ""che"" nella frase seguente: Ieri ho conosciuto Anna. Che persona interessante!"
"Il ""che"" in questa frase è un pronome relativo che introduce una subordinata relativa, ovvero la frase ""persona interessante"". Quindi il ""che"" funziona come congiunzione subordinante."
---------------------------------------------------------Domanda 21----------------------------------------------------------------------------------------------
1"Indica quale funziona ha il ""che"" nella frase seguente: Che film hai visto quanto sei andato al cinema?"
"Il ""che"" è una congiunzione che introduce una proposizione subordinata, quindi la risposta è ""hai visto""."
---------------------------------------------------------Domanda 22----------------------------------------------------------------------------------------------
"Indica quale funziona ha il ""che"" nella frase seguente: Avevo così tanto sonno che mi sono addormentato sul divano."
"Il ""che"" è una congiunzione coordinante."
"""

2. DATA PARSING (Cleaning the text)

In [ ]:
def parse_dataset(text_data, q_type="type1"):
    questions = []
    chunks = re.split(r'-{10,}Domanda \d+-{10,}', text_data)
    for chunk in chunks:
        chunk = chunk.strip()
        if not chunk: continue
        lines = [l.strip() for l in chunk.split('\n') if l.strip()]

        if q_type == "type1": # Multiple Choice
            # Find the answer line (heuristic)
            answer_line = next((l for l in lines if l.startswith('"') or l.startswith("La frase") or l.startswith("b.") or l.startswith("a.")), lines[-1])
            question_text = "\n".join([l for l in lines if l != answer_line])
            ground_truth = answer_line.replace('"', '').strip()
        else: # Analysis
            question_text = lines[0]
            if len(lines) > 1:
                ground_truth = " ".join(lines[1:]).replace('"', '').strip()
            else:
                ground_truth = "N/A"

        questions.append({"question": question_text, "ground_truth": ground_truth})
    return questions

# Combine Datasets
dataset = parse_dataset(RAW_TEXT_1, "type1") + parse_dataset(RAW_TEXT_2, "type2")
print(f"✅ Extracted {len(dataset)} questions from raw files.")

3. MINERVA GENERATION

In [ ]:
MODEL_ID = "sapienzanlp/Minerva-7B-instruct-v1.0"
print("⏳ Loading Minerva Model (4-bit)...")

quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    model = AutoModelForCausalLM.from_pretrained(MODEL_ID, quantization_config=quant_config, device_map="auto")

    pipe = pipeline(
        "text-generation", model=model, tokenizer=tokenizer,
        max_new_tokens=150, temperature=0.1, do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

    final_data = []
    print("🚀 Generating answers...")

    for i, item in enumerate(dataset):
        prompt = f"Sei un esperto di grammatica italiana.\nDomanda: {item['question']}\nRisposta:"
        try:
            res = pipe(prompt)[0]['generated_text']
            minerva_ans = res.replace(prompt, "").split("Domanda:")[0].strip()
        except:
            minerva_ans = "Error"

        print(f"[{i+1}/{len(dataset)}] Generated.")
        final_data.append({
            "id": i+1,
            "question": item['question'],
            "ground_truth": item['ground_truth'],
            "answer_minerva": minerva_ans
        })

    with open("minerva_qna.jsonl", "w", encoding="utf-8") as f:
        for entry in final_data:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print("\n✅ Step 2 Complete! Answers saved to 'minerva_qna.jsonl'.")
    print("⚠️ NOW: Go to Runtime > Disconnect and delete runtime, then run Cell 3.")

except Exception as e:
    print(f"Error: {e}")

# Phase3. Judge Evaluation (Qwen)

In [ ]:
import os, json, csv
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

In [ ]:
INPUT_JSONL = "minerva_qna.jsonl"
OUT_CSV = "thesis_evaluation_results.csv"
JUDGE_MODEL = "Qwen/Qwen2.5-7B-Instruct"

EVAL_PROMPT = """Sei un professore universitario di linguistica italiana.
Confronta la risposta del modello AI con la risposta corretta (Ground Truth).

Domanda: {question}
Risposta Corretta: {reference}
Risposta AI: {candidate}

Assegna un punteggio da 1 a 5 (5 = perfetta).
Rispondi SOLO in JSON: {{"score": <int>, "reasoning": "<text>"}}
"""

if not os.path.exists(INPUT_JSONL):
    print("❌ Error: 'minerva_qna.jsonl' missing. Run Cell 2 first.")
else:
    print(f"⚖️ Loading Judge: {JUDGE_MODEL}...")
    quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

    try:
        tokenizer = AutoTokenizer.from_pretrained(JUDGE_MODEL, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(JUDGE_MODEL, quantization_config=quant_config, device_map="auto", trust_remote_code=True)
        judge_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=250)

        with open(INPUT_JSONL, 'r', encoding='utf-8') as f:
            rows = [json.loads(line) for line in f]

        results = []
        print(f"📊 Evaluating {len(rows)} items...")

        for row in rows:
            prompt_text = EVAL_PROMPT.format(
                question=row['question'],
                reference=row['ground_truth'],
                candidate=row['answer_minerva']
            )
            messages = [{"role": "user", "content": prompt_text}]
            formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

            try:
                out = judge_pipe(formatted_prompt, do_sample=False, temperature=0.1)[0]['generated_text']
                json_part = out.split(formatted_prompt)[-1].strip()
                if "```json" in json_part: json_part = json_part.split("```json")[1].split("```")[0].strip()
                elif "{" in json_part: json_part = json_part[json_part.find("{"):json_part.rfind("}")+1]

                data = json.loads(json_part)
                score, reason = data.get("score", 1), data.get("reasoning", "Parse Error")
            except:
                score, reason = 1, "Error"

            print(f"Q{row['id']}: Score {score}/5")
            results.append({
                "id": row['id'],
                "question": row['question'][:50],
                "ground_truth": row['ground_truth'],
                "minerva_answer": row['answer_minerva'],
                "score": score,
                "reason": reason
            })

        keys = results[0].keys()
        with open(OUT_CSV, 'w', newline='', encoding='utf-8') as f:
            w = csv.DictWriter(f, keys)
            w.writeheader()
            w.writerows(results)

        print(f"\n✅ Done! Download '{OUT_CSV}' from the files tab.")

    except Exception as e:
        print(f"Error: {e}")